<a href="https://colab.research.google.com/github/hwankang/Defense/blob/main/0705_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
import pandas as pd
data=pd.read_csv('005930.KS.csv', date_parser=True)
data.head()

In [ ]:
import numpy as np  
import datetime # 날짜,시간 활용
import matplotlib.pyplot as plt #데이터 시각화

# 딥러닝을 위한 라이브러리 keras 
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
#[출처] [Quant] LSTM 시계열 예측모델 이용하여 주가 예측하기 (딥러닝)|작성자 Investgy

In [ ]:
high_prices = data['High'].values
low_prices = data['Low'].values
mid_prices = (high_prices + low_prices) / 2

len(mid_prices)

In [ ]:
seq_len = 20  #최근 20일치 데이터를 보고 다음을 예측할 것임
sequence_length = seq_len + 1

results = []
for index in range(len(mid_prices) - sequence_length): 
    results.append(mid_prices[index: index + sequence_length]) 

In [ ]:
normalized_data = [] 

# 윈도우를 하나씩 훑어가면서 정규화하기 
for window in results: 
    normalized_window = [(((float(p)+0.001) / (float(window[0])+0.001) ) - 1) for p in window] #윈도우 내부에 있는 값들을 첫번째 값으로 나누어서 정규화 
    normalized_data.append(normalized_window)

In [ ]:
norm_result = np.array(normalized_data)
norm_result

In [ ]:
# split train and test data
row = int(round(norm_result.shape[0] * 0.9)) #전체 개수 중 90%를 train으로 사용할 것임 
train = norm_result[:row, :]
np.random.shuffle(train) #랜덤으로 섞는다 

# train data
x_train = train[:, :-1]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = train[:, -1]

# test data
x_test = norm_result[row:, :-1]
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
y_test = norm_result[row:, -1] 


x_train.shape, x_test.shape #1194일의 주식 가격을 통해 133일의 주식 가격을 예측해보기

In [ ]:
model = Sequential() 
model.add(LSTM(50, return_sequences=True, input_shape=(50, 1))) #첫번째 LSTM 은 유닛수 50개
model.add(LSTM(64, return_sequences=False)) #두번째 LSTM 유닛수는 64개
model.add(Dense(1, activation='linear')) # 아웃풋으로 나오는 값은 1개 (다음날 하루 예측)
model.compile(loss='mse', optimizer='rmsprop') #손실 함수 
model.summary() #모델의 개요

In [ ]:
model.fit(x_train, y_train,
    validation_data=(x_test, y_test),
    batch_size=10,
    epochs=20) 

In [ ]:
predictd = model.predict(x_test) #예측하기

fig = plt.figure(facecolor='white', figsize=(20, 10))
axes = fig.add_subplot(111)
axes.plot(y_test, label='CloseData', color = 'midnightblue')
axes.plot(predictd, label='Prediction', color = 'lightsteelblue')
axes.legend()
plt.show()